Test difference in synapse numbers per column between "fetch_neurons" and query. 

In [ ]:
from pathlib import Path
import sys
import os
from dotenv import find_dotenv
PROJECT_ROOT = Path(find_dotenv()).parent
sys.path.append(str(PROJECT_ROOT.joinpath('src')))
from neuprint import Client, fetch_neurons, NeuronCriteria as NC, fetch_custom
from neuview.config import Config
config = Config.load(PROJECT_ROOT / 'config.yaml')
c = Client(config.neuprint.server, dataset=config.neuprint.dataset, token=os.environ['NEUPRINT_TOKEN'])

When printing the "row" entry for "ME_col_01_07" in `PageGenerator._get_col_layer_values`:

""" 

# Convert to list of dictionaries for template
        column_summary = []
        for _, row in neurons_per_column.iterrows():
            key = (row['region'], row['side'], row['hex1_dec'], row['hex2_dec'])
            if (row['region'], row['side'], row['hex1_dec'], row['hex2_dec']) == ("ME", "R", 1, 7):
                print(row)

"""

It returns:

region                                                                  ME<br>
side                                                                     R<br>
hex1_dec                                                                 1<br>
hex2_dec                                                                 7<br>
bodyId                                                                   4<br>
pre                                                                      4<br>
post                                                                    74<br>
total                                                                   78<br>
mean_pre_per_neuron                                                    1.0<br>
mean_post_per_neuron                                                  18.5<br>
mean_total_per_neuron                                                 19.5<br>
synapses_list                            [0, 1, 0, 16, 57, 0, 0, 0, 0, 32]<br>
neurons_list                                [0, 1, 0, 1, 4, 0, 0, 0, 0, 1]<br>
synapse_colors           [#ffffff, #fee5d9, #ffffff, #fee5d9, #fee5d9, ...<br>
neuron_colors            [#ffffff, #fee5d9, #ffffff, #fee5d9, #fcbba1, ...<br>
synapses_total                                                       106.0<br>

So, from fetch_neurons - - 4 pre + 74 post synapses within this column for TmY3. Therefore a total of 78 synapses. <br>
But from the query there are a total of 106 synapses in this column. 


1. What is returned from "fetch_neurons"

In [ ]:
type_neuron = "TmY3"
[neuron_df, roi_df] = fetch_neurons(NC(type=type_neuron))

In [ ]:
bodyId = 68242

In [ ]:
roi_df[roi_df['roi']=="ME_R_col_01_07"]

In [ ]:
## From "fetch_neurons" - - - roi_df

# Across all bodyIds:
# sum pre = 4
# sum post = (5+49+6+14) = 74
# sum all = 78

# For the bodyId 68242:
# pre = 4
# post = 49
# total = 53

2. What is returned from the query used in `PageGenerator._get_col_layer_values`. (Added returning the number of "pre/post" specifically.)

In [ ]:
# Query in PageGenerator
query = fr"""
        MATCH (n:Neuron)-[:Contains]->(nss:SynapseSet)-[:Contains]->(ns:Synapse)
        WHERE n.type = '{type_neuron}'
        WITH ns,
        CASE
            WHEN exists(ns['ME(R)'])  THEN ['ME','R']
            WHEN exists(ns['ME(L)'])  THEN ['ME','L']
            WHEN exists(ns['LO(R)'])  THEN ['LO','R']
            WHEN exists(ns['LO(L)'])  THEN ['LO','L']
            WHEN exists(ns['LOP(R)']) THEN ['LOP','R']
            WHEN exists(ns['LOP(L)']) THEN ['LOP','L']
        END AS layerKey
        WITH ns, layerKey
        WHERE layerKey IS NOT NULL
        RETURN
        ns.olHex1 AS hex1_dec,
        ns.olHex2 AS hex2_dec,
        ns.olLayer AS layer,
        layerKey[0] AS region,
        layerKey[1] AS side,
        count(ns) AS total_synapses,
        sum(CASE WHEN ns.type = 'pre'  THEN 1 ELSE 0 END)  AS n_pre,
        sum(CASE WHEN ns.type = 'post' THEN 1 ELSE 0 END)  AS n_post,
        ns.bodyId AS bodyId,
        count(DISTINCT ns.bodyId) AS neuron_count
        ORDER BY hex1_dec, hex2_dec, layer
        """
df = c.fetch_custom(query)
df = df.dropna(subset=['layer'])
df['layer'] = df['layer'].astype(int)

In [ ]:
# For all bodyIds within the column. 
df_sub1 = df[
    (df['region'] == "ME") &
    (df['side'] == "R") &
    (df['hex1_dec'] == 1) &
    (df['hex2_dec'] == 7)
]
df_sub1

In [ ]:
# Totals across all bodyIds within the column. 
df_sub1['n_post'].sum() # 74
df_sub1['n_pre'].sum() # 32
df_sub1['total_synapses'].sum() # 106

In [ ]:
# For one bodyId

df_sub = df[
    (df['bodyId'] == bodyId) &
    (df['region'] == "ME") &
    (df['side'] == "R") &
    (df['hex1_dec'] == 1) &
    (df['hex2_dec'] == 7)
]
df_sub

In [ ]:
# Total synapses for just the one boydId:

# sum total - - 16+32+33 = 81
df_sub['total_synapses'].sum()

# sum pre - - 25+7  = 32
df_sub['n_pre'].sum()

# sum post - - - 16+26+7 = 49
df_sub['n_post'].sum()

For column "ME_col_01_07"

From "fetch_neurons":
    pre = 4
    post = (5+49+6+14) = 74
    all = 78

From the query:
    pre = 32
    post = 74
    all = 106

- - - > fetch_neurons seems to not count all of the 'pre' synapses... 